In [7]:
pip install ace_tools


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np

# Dataset generation function
def generate_large_synthetic_data(samples_per_stage=150):
    stages = ["Awake", "Light", "Deep", "REM"]
    data = []

    for stage in stages:
        # Generate synthetic heart rate (HR) data
        hr = np.random.normal(70 if stage == "Awake" else 60 if stage == "Light" else 50 if stage == "Deep" else 65, 10, samples_per_stage)
        
        # Generate synthetic accelerometer (ACC) data
        acc = np.random.normal(1 if stage == "Awake" else 0.5 if stage == "Light" else 0.2 if stage == "Deep" else 0.3, 0.2, samples_per_stage)
        
        # Introduce random variations to simulate real-world sensor noise
        transition_points = np.random.choice(samples_per_stage - 5, int(0.1 * samples_per_stage), replace=False)
        for tp in transition_points:
            hr[tp:tp + 5] += np.random.normal(5, 2, 5)
            acc[tp:tp + 5] += np.random.normal(0.1, 0.05, 5)
        
        # Combine into a DataFrame
        stage_data = pd.DataFrame({
            "heart_rate": hr,
            "accelerometer": acc,
            "stage": [stage] * samples_per_stage
        })
        data.append(stage_data)

    return pd.concat(data, ignore_index=True)

# Generate dataset
data = generate_large_synthetic_data()

# Display the entire dataset in a notebook environment
##import ace_tools as tools; tools.display_dataframe_to_user(name="Generated Sleep Activity Dataset", dataframe=data)





In [3]:
# Display the full dataset
import pandas as pd

# Ensure all rows and columns are visible in the notebook
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Display the dataset
print("Dataset Shape:", data.shape)  # Print the shape of the dataset
print(data)  # Print the dataset content



Dataset Shape: (600, 3)
     heart_rate  accelerometer  stage
0     73.331580       1.059298  Awake
1     68.253495       0.929562  Awake
2     87.923379       0.943500  Awake
3     86.926818       1.133956  Awake
4     78.941390       0.848318  Awake
5     85.371622       1.352253  Awake
6     69.005387       1.157244  Awake
7     68.661631       1.178648  Awake
8    107.657456       1.535010  Awake
9     66.705837       1.434847  Awake
10    98.109032       0.880834  Awake
11    72.368284       0.813972  Awake
12    66.710617       1.098789  Awake
13    53.014633       1.117695  Awake
14    72.177611       0.933898  Awake
15    88.237048       1.375299  Awake
16    62.304552       0.947451  Awake
17    81.669298       1.166856  Awake
18    68.540471       0.977605  Awake
19    68.106260       0.679478  Awake
20    87.033141       1.079887  Awake
21    52.684935       0.783834  Awake
22    72.090304       1.359814  Awake
23    64.975734       0.977000  Awake
24    73.907128       0.85

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Step 1: Preprocess the Dataset
def preprocess_data(data, sequence_length=50):
    # Encode the labels
    label_encoder = LabelEncoder()
    data['stage'] = label_encoder.fit_transform(data['stage'])
    unique_classes = label_encoder.classes_

    # Extract features and labels
    X = data[['heart_rate', 'accelerometer']].values
    y = data['stage'].values

    # Normalize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Create sequences for LSTM
    X_seq, y_seq = [], []
    for i in range(len(X_scaled) - sequence_length):
        X_seq.append(X_scaled[i:i + sequence_length])
        y_seq.append(y[i + sequence_length])

    return np.array(X_seq), np.array(y_seq), scaler, label_encoder


In [6]:
# Preprocess the generated dataset
sequence_length = 50
X_seq, y_seq, scaler, label_encoder = preprocess_data(data, sequence_length)

# Step 2: Split the Dataset
X_train, X_temp, y_train, y_temp = train_test_split(X_seq, y_seq, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training data shape: {X_train.shape}, Validation data shape: {X_val.shape}, Testing data shape: {X_test.shape}")

Training data shape: (385, 50, 2), Validation data shape: (82, 50, 2), Testing data shape: (83, 50, 2)


In [7]:
from tensorflow.keras.layers import Input

def build_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),  # Use Input as the first layer
        LSTM(64, return_sequences=True),
        Dropout(0.2),
        LSTM(32, return_sequences=False),
        Dense(16, activation='relu'),
        Dense(5, activation='softmax')  # 5 sleep stages
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [8]:
# Build and train the LSTM model
model = build_lstm((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Display validation accuracy during training
for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
        history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy'])):
    print(f"Epoch {epoch + 1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

# Save model in Keras format and scaler
model.save("sleep_lstm_model_combined.keras")
np.save("scaler_mean.npy", scaler.mean_)
np.save("scaler_scale.npy", scaler.scale_)

print("Model and scaler saved successfully.")



Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - accuracy: 0.3026 - loss: 1.5826 - val_accuracy: 0.6627 - val_loss: 1.3731
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6487 - loss: 1.3268 - val_accuracy: 0.7229 - val_loss: 0.9875
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6686 - loss: 0.9690 - val_accuracy: 0.9277 - val_loss: 0.7130
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9338 - loss: 0.6655 - val_accuracy: 0.9639 - val_loss: 0.4508
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9236 - loss: 0.4918 - val_accuracy: 0.9639 - val_loss: 0.3316
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9369 - loss: 0.3490 - val_accuracy: 0.9639 - val_loss: 0.2686
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.9434 - loss: 0.3039 - val_accuracy: 0.9639 - val_loss: 0.2207
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9522 - loss: 0.2433 - val_accuracy: 0.9759 - v

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


# Evaluate the model on the test set
print("\nEvaluating the model on the test set...")
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=32)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Generate predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # Convert one-hot probabilities to class indices

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))




Evaluating the model on the test set...
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9663 - loss: 0.1679
Test Loss: 0.1665, Test Accuracy: 0.9639
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 184ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        20
           1       0.95      0.95      0.95        22
           2       0.95      0.95      0.95        20
           3       1.00      0.95      0.98        21

    accuracy                           0.96        83
   macro avg       0.96      0.96      0.96        83
weighted avg       0.96      0.96      0.96        83



In [17]:
import tensorflow as tf

# Enable TensorFlow Ops for unsupported operations
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Default TFLite operations
    tf.lite.OpsSet.SELECT_TF_OPS    # TensorFlow operations
]

# Disable experimental lowering of tensor list operations
converter._experimental_lower_tensor_list_ops = False

# Convert the model
try:
    tflite_model = converter.convert()
    with open("sleep_lstm_model_with_tf_ops.tflite", "wb") as f:
        f.write(tflite_model)
    print("Model converted with TensorFlow ops and saved as 'sleep_lstm_model_with_tf_ops.tflite'")
except Exception as e:
    print(f"Error during conversion: {e}")


NameError: name 'converter' is not defined

In [19]:
# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="sleep_lstm_model_with_tf_ops.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test with dummy input (adjust shape to match your model)
import numpy as np
dummy_input = np.random.rand(1, 50, 2).astype(np.float32)  # Replace with actual input shape

# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], dummy_input)

# Run inference
interpreter.invoke()

# Get the output
output = interpreter.get_tensor(output_details[0]['index'])
print("Model Output:", output)


Model Output: [[0.02760666 0.00528242 0.95371723 0.00883773 0.00455591]]


In [21]:
import os
import tensorflow as tf

tflite_model_path = "sleep_lstm_model_with_tf_ops.tflite"  # Ensure the correct path is specified

# Check if the file exists
if not os.path.exists(tflite_model_path):
    raise FileNotFoundError(f"The specified TFLite model file does not exist: {tflite_model_path}")

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()
print("TFLite model loaded successfully!")



TFLite model loaded successfully!


In [23]:
# Paths
tflite_model_path = "sleep_lstm_model_with_tf_ops.tflite"  # Path to your TFLite model
tfjs_output_dir = "tfjs_model"  # Directory to save the TensorFlow.js model

# Use TensorFlow.js converter to directly convert the TFLite model to a JavaScript format
import os
os.system(f"tensorflowjs_converter --input_format=tflite --output_format=tfjs_graph_model {tflite_model_path} {tfjs_output_dir}")

print(f"TensorFlow.js model has been saved to: {tfjs_output_dir}")


TensorFlow.js model has been saved to: tfjs_model


In [25]:
pip install tensorflowjs


Note: you may need to restart the kernel to use updated packages.


In [27]:
tensorflowjs_converter --version


NameError: name 'tensorflowjs_converter' is not defined

In [29]:
import os

# Check the version of tensorflowjs_converter
os.system("tensorflowjs_converter --version")


1

In [31]:
pip show tensorflowjs


Name: tensorflowjs
Version: 3.18.0
Summary: 
Home-page: https://js.tensorflow.org/
Author: Google LLC
Author-email: opensource@google.com
License: Apache 2.0
Location: D:\Anaconda\Lib\site-packages
Requires: packaging, six, tensorflow, tensorflow-hub
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [1]:
import subprocess

# Run tensorflowjs_converter to check the version
subprocess.run(["tensorflowjs_converter", "--version"])


CompletedProcess(args=['tensorflowjs_converter', '--version'], returncode=1)

In [3]:
where tensorflowjs_converter


SyntaxError: invalid syntax (2234867930.py, line 1)

In [5]:
import subprocess

result = subprocess.run(
    ["tensorflowjs_converter", "--version"],
    capture_output=True,
    text=True
)
print("Return Code:", result.returncode)
print("Standard Output:", result.stdout)
print("Error Output:", result.stderr)


Return Code: 1
Standard Output: 
Error Output: 2025-01-26 14:20:30.401951: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-26 14:20:31.931666: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "D:\Anaconda\Scripts\tensorflowjs_converter.exe\__main__.py", line 4, in <module>
  File "D:\Anaconda\Lib\site-packages\tensorflowjs\__init__.py", line 21, in <module>
    from tensorflowjs import converters
  File "D:

In [7]:
pip install tensorflowjs


In [9]:
tensorflowjs_converter --input_format=tflite --output_format=tfjs_graph_model sleep_lstm_model_with_tf_ops.tflite tfjs_model


SyntaxError: cannot assign to expression (2112401995.py, line 1)

In [11]:
import os

# Run the command using os.system
os.system("tensorflowjs_converter --input_format=tflite --output_format=tfjs_graph_model sleep_lstm_model_with_tf_ops.tflite tfjs_model")

# Alternatively, using subprocess
import subprocess

subprocess.run(["tensorflowjs_converter",
                "--input_format=tflite",
                "--output_format=tfjs_graph_model",
                "sleep_lstm_model_with_tf_ops.tflite",
                "tfjs_model"])


CompletedProcess(args=['tensorflowjs_converter', '--input_format=tflite', '--output_format=tfjs_graph_model', 'sleep_lstm_model_with_tf_ops.tflite', 'tfjs_model'], returncode=1)

In [13]:
ls -l sleep_lstm_model_with_tf_ops.tflite


 Volume in drive C is OS
 Volume Serial Number is 5209-F365

 Directory of C:\Users\girid


 Directory of C:\Users\girid

01/25/2025  11:10 PM            60,272 sleep_lstm_model_with_tf_ops.tflite
               1 File(s)         60,272 bytes
               0 Dir(s)  123,316,383,744 bytes free


In [15]:
import tensorflow as tf
interpreter = tf.lite.Interpreter(model_path="sleep_lstm_model_with_tf_ops.tflite")
interpreter.allocate_tensors()
print("TFLite model is valid and ready for conversion.")


TFLite model is valid and ready for conversion.


In [17]:
pip show tensorflow


Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: D:\Anaconda\Lib\site-packages
Requires: tensorflow-intel
Required-by: tensorflowjs
Note: you may need to restart the kernel to use updated packages.


In [19]:
import subprocess

result = subprocess.run(
    ["tensorflowjs_converter", "--input_format=tflite", "--output_format=tfjs_graph_model", "sleep_lstm_model_with_tf_ops.tflite", "tfjs_model"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

print("Return Code:", result.returncode)
print("Standard Output:", result.stdout)
print("Error Output:", result.stderr)


Return Code: 1
Standard Output: 
Error Output: 2025-01-27 01:19:57.555278: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-27 01:19:59.173258: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "D:\Anaconda\Scripts\tensorflowjs_converter.exe\__main__.py", line 4, in <module>
  File "D:\Anaconda\Lib\site-packages\tensorflowjs\__init__.py", line 21, in <module>
    from tensorflowjs import converters
  File "D:

In [21]:
tensorflowjs_converter --input_format=keras sleep_lstm_model_combined.keras tfjs_model


SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (2056763468.py, line 1)